# BUILDING MACHINE LEARNING MODEL

In [14]:
# Import thư viện
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

import torch
import torch.nn as nn
import torch.optim as optim


In [4]:
# đọc dữ liệu
df = pd.read_csv("/Users/nguyengiabao/Documents/house_price/data/Pretraining_data.csv")
df

,Total Floor,Bathroom,Price,ImputedCarpet_Area,ImputedCurrent_Floor
0,11.0,1.0,6000.0,500.0,10.0
1,22.0,2.0,13799.0,473.0,3.0
2,29.0,2.0,17500.0,779.0,10.0
3,42.0,2.0,18824.0,635.0,20.0
4,7.0,1.0,6618.0,1100.0,2.0
...,...,...,...,...,...
169079,4.0,3.0,3225.0,1100.0,2.0
169080,6.0,3.0,3274.0,1100.0,4.0
169081,3.0,3.0,4343.0,1250.0,1.0
169082,2.0,2.0,4231.0,1100.0,2.0


In [5]:
# Select features and labels
def select_features(df, features_name):   # hàm chọn các đặc trưng
    return df[features_name].values

def select_labels(df, labels_name):       # hàm chọn biến mục tiêu
    return df[labels_name].values

In [7]:
# Select features
features = select_features(df, ["Total Floor", "Bathroom", "ImputedCarpet_Area", "ImputedCurrent_Floor"])
# Select labels
labels = select_labels(df, "Price")


In [8]:
features

array([[1.10e+01, 1.00e+00, 5.00e+02, 1.00e+01],
       [2.20e+01, 2.00e+00, 4.73e+02, 3.00e+00],
       [2.90e+01, 2.00e+00, 7.79e+02, 1.00e+01],
       ...,
       [3.00e+00, 3.00e+00, 1.25e+03, 1.00e+00],
       [2.00e+00, 2.00e+00, 1.10e+03, 2.00e+00],
       [1.30e+01, 4.00e+00, 1.10e+03, 5.00e+00]], shape=(169084, 4))

In [9]:
labels

array([ 6000., 13799., 17500., ...,  4343.,  4231.,  6162.],
      shape=(169084,))

In [ ]:
# tách tập dữ liệu thành tập dữ liệu huấn luyện, kiểm định và kiểm tra
feature_train, feature_test, label_train, label_test = train_test_split(features, labels, test_size=0.1, random_state=42)
feature_train, feature_val, label_train, label_val = train_test_split(features, labels, test_size=0.1, random_state=42)

In [13]:
# build model 
model = LinearRegression()
model.fit(feature_train, label_train)  # Gradient Descent

train_r2_score = model.score(feature_train, label_train)
val_r2_score = model.score(feature_val, label_val)

print(f"R2 score of train set: {train_r2_score}")
print(f"R2 score of val set: {val_r2_score}")

# hàm dự đoán giá nhà
def predict_price(floor1, floor2, bathroom, carpet_area):
    input_data = np.array([[floor1, floor2, bathroom, carpet_area]])
    predicted_price = model.predict(input_data)[0]
    return predicted_price

# Dự đoán giá nhà với dữ liệu cụ thể
sample_prediction = predict_price(5, 2, 2, 750)
print(f"Giá nhà dự đoán: {sample_prediction:.2f} rupees")

R2 score of train set: 0.0038710115871827266
R2 score of val set: 0.12803737180740082
Giá nhà dự đoán: 23658.98 rupees


# BUILDING DEEP LEARNING MODEL

In [ ]:
# xây dựng mô hình
class HousePriceModel(nn.Module):
    def __init__(self):
        super(HousePriceModel, self).__init__()
        self.fc1 = nn.Linear(4, 64)   # 4 đặc trưng → 64 nơ-ron ẩn
        self.fc2 = nn.Linear(64, 32)  # 64 → 32 nơ-ron ẩn
        self.fc3 = nn.Linear(32, 1)   # 32 → 1 đầu ra (giá nhà)

    def forward(self, x):
        x = torch.relu(self.fc1(x))  # Kích hoạt ReLU tầng 1
        x = torch.relu(self.fc2(x))  # Kích hoạt ReLU tầng 2
        x = self.fc3(x)  # Đầu ra không có activation (hàm tuyến tính)
        return x


In [18]:
# Chuẩn bị dữ liệu
# Chọn các cột làm đặc trưng (features)
X = df[["Total Floor", "Bathroom", "ImputedCarpet_Area", "ImputedCurrent_Floor"]].values  

# Chọn cột giá nhà làm nhãn (labels)
y = df[["Price"]].values  

# Chia thành tập huấn luyện và kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Chuyển đổi dữ liệu sang tensor của PyTorch
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

In [ ]:
# khởi tạo mô hình, hàm mất mát và bộ tối ưu
model = HousePriceModel()
criterion = nn.MSELoss()  # Mean Squared Error (MSE) dùng cho bài toán hồi quy
optimizer = optim.Adam(model.parameters(), lr=0.01)  # Adam optimizer


In [20]:
# huấn luyện mô hình
num_epochs = 500  # Số lần lặp huấn luyện

for epoch in range(num_epochs):
    # Forward: dự đoán
    y_pred = model(X_train_tensor)
    
    # Tính toán loss (MSE)
    loss = criterion(y_pred, y_train_tensor)
    
    # Backward: tính gradient và cập nhật trọng số
    optimizer.zero_grad()  # Xóa gradient cũ
    loss.backward()  # Lan truyền ngược (backpropagation)
    optimizer.step()  # Cập nhật trọng số
    
    # In loss mỗi 50 epochs
    if (epoch+1) % 50 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")


Epoch [50/500], Loss: 983736832.0000
Epoch [100/500], Loss: 983326912.0000
Epoch [150/500], Loss: 982842112.0000
Epoch [200/500], Loss: 978174464.0000
Epoch [250/500], Loss: 951990848.0000
Epoch [300/500], Loss: 949816192.0000
Epoch [350/500], Loss: 952135104.0000
Epoch [400/500], Loss: 971475776.0000
Epoch [450/500], Loss: 964977216.0000
Epoch [500/500], Loss: 954750976.0000


In [21]:
# kiểm tra mô hình
# Chuyển sang chế độ đánh giá (không cập nhật trọng số)
model.eval()

# Dự đoán trên tập kiểm tra
with torch.no_grad():
    y_test_pred = model(X_test_tensor)

# In một số dự đoán
print("Actual prices:", y_test[:5].flatten())
print("Predicted prices:", y_test_pred[:5].flatten())


Actual prices: [ 8000.  4472.  5655. 12594. 11111.]
Predicted prices: tensor([6914.1450,  858.9201, 3589.0742, 6195.7627,  490.2827])
